In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd
import os

# Path to the folder containing CSV files
folder_path = '/content/drive/My Drive/Preprocessed'

# Combine all CSV files into one DataFrame
all_data = pd.DataFrame()

for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):  # Ensure it's a CSV file
        file_path = os.path.join(folder_path, file_name)
        # Specify the semicolon as the delimiter
        temp_data = pd.read_csv(file_path, sep=';')
        all_data = pd.concat([all_data, temp_data], ignore_index=True)

# Display the combined dataset
print("Combined DataFrame:")
print(all_data.info())
print(all_data.head())


In [ ]:
import pandas as pd

# Load one of the CSV files for inspection
file_path = '/content/drive/My Drive/Preprocessed/HUPA0015P.csv'  # Replace with the actual path
data = pd.read_csv(file_path, sep=';', header=None)  # Use sep=';' to handle semicolon delimiter

# Display the first few rows
print(data.head())

In [ ]:
data = pd.read_csv(file_path, sep=';', header=0, engine='python')  # Use 'python' engine for better compatibility
print(data.columns)
print(data.head())


In [ ]:
import os

# Path to the folder
folder_path = '/content/drive/My Drive/Preprocessed'

all_data = pd.DataFrame()

for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)

        # Read and split the data
        temp_data = pd.read_csv(file_path, header=None)
        temp_data_split = temp_data[0].str.split(';', expand=True)
        temp_data_split.columns = ['time', 'glucose', 'calories', 'heart_rate', 'steps', 'basal_rate', 'bolus_volume_delivered', 'carb_input']

        # Combine into the main DataFrame
        all_data = pd.concat([all_data, temp_data_split], ignore_index=True)

print(all_data.info())
print(all_data.head())


In [ ]:

import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Combine all preprocessed CSV files
preprocessed_dir = '/content/drive/My Drive/Preprocessed'
all_data = pd.DataFrame()

for file_name in os.listdir(preprocessed_dir):
    file_path = os.path.join(preprocessed_dir, file_name)
    # Read and split the data, assuming semicolon delimiter and no header
    temp_data = pd.read_csv(file_path, header=None)
    temp_data_split = temp_data[0].str.split(';', expand=True)
    temp_data_split.columns = ['time', 'glucose', 'calories', 'heart_rate', 'steps', 'basal_rate', 'bolus_volume_delivered', 'carb_input']
    all_data = pd.concat([all_data, temp_data_split], ignore_index=True)
# Handle missing values
all_data = all_data.dropna()

# Separate features and target
target_column = 'glucose'  # Replace with your actual target column name
X = all_data.drop(columns=[target_column, 'time'])  # Drop 'time' column from features
y = all_data[target_column]

# Convert columns to numeric, errors='coerce' to handle potential non-numeric values
X = X.apply(pd.to_numeric, errors='coerce')

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, Input, MaxPooling1D

# Reshape data for DCNN
X_train_dcnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_dcnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define the DCNN model
dcnn_model = Sequential([
    Input(shape=(X_train_dcnn.shape[1], 1)),
    Conv1D(filters=32, kernel_size=3, dilation_rate=1, activation='relu', padding='same'),  # Added padding='same'
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, dilation_rate=1, activation='relu', padding='same'),  # Added padding='same'
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compile and train the model
dcnn_model.compile(optimizer=tf.keras.optimizers.Adam(clipvalue=1.0), loss='mean_squared_error', metrics=['mae'])
history = dcnn_model.fit(X_train_dcnn, y_train, epochs=50, batch_size=32, validation_data=(X_test_dcnn, y_test))


In [ ]:
# Make predictions on the test set
predictions= dcnn_model.predict(X_test_dcnn)

In [ ]:
print("Shape of X_test_dcnn:", X_test_dcnn.shape)

In [ ]:
import numpy as np

# Check for NaN or infinite values in X_test_dcnn
print("NaN in X_test_dcnn:", np.isnan(X_test_dcnn).any())
print("Infinite in X_test_dcnn:", np.isinf(X_test_dcnn).any())

# Check for NaN or infinite values in y_test
print("NaN in y_test:", np.isnan(y_test).any())
print("Infinite in y_test:", np.isinf(y_test).any())


In [ ]:
X_test_dcnn = np.nan_to_num(X_test_dcnn)  # Replace NaN with 0
y_test = np.nan_to_num(y_test)

In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, Input, MaxPooling1D, Dropout, LSTM, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Replace NaN in predictions and y_test with 0
predictions = np.nan_to_num(predictions)
y_test = np.nan_to_num(y_test)

# Assuming 'predictions' are your DCNN model's predictions and 'y_test' are the true values
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R2): {r2}")

In [ ]:
# ... (Your DCNN model code) ...

# Make predictions on the test set
predictions_dcnn = dcnn_model.predict(X_test_dcnn)

# Define thresholds for categorization
thresholds = [70, 140]  # Example thresholds

# Categorize predictions and true values
y_test_cat = np.digitize(y_test, thresholds)
predictions_dcnn_cat = np.digitize(predictions_dcnn, thresholds)

# Calculate confusion matrix
cm_dcnn = confusion_matrix(y_test_cat, predictions_dcnn_cat)

# Calculate metrics
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
precision_dcnn = precision_score(y_test_cat, predictions_dcnn_cat, average='weighted')
recall_dcnn = recall_score(y_test_cat, predictions_dcnn_cat, average='weighted')
accuracy_dcnn = accuracy_score(y_test_cat, predictions_dcnn_cat)
f1_dcnn = f1_score(y_test_cat, predictions_dcnn_cat, average='weighted')

# Print metrics
print("DCNN Metrics:")
print("Confusion Matrix:", cm_dcnn)
print("Precision:", precision_dcnn)
print("Recall:", recall_dcnn)
print("Accuracy:", accuracy_dcnn)
print("F1-score:", f1_dcnn)